In [7]:
import cv2
import numpy as np
import sys
from IPython.display import clear_output

In [ ]:
#Q1

canvas = np.ones((500, 500, 3), dtype=np.uint8) * 255

# Define colors in BGR (OpenCV uses BGR)
red = (66, 66, 255)
green = (87, 215, 126)
blue = (255, 153, 0)
white = (255, 255, 255)

# Define circle centers 
red_center = (250, 150)
green_center = (170, 300)
blue_center = (330, 300)

outer_radius = 70
inner_radius = 35


def draw_opencv_ring(img, center, color, notch_angle):
    cv2.circle(img, center, outer_radius, color, -1)
    cv2.circle(img, center, inner_radius, white, -1)
    cv2.ellipse(img, center, (outer_radius, outer_radius), 0,
                notch_angle - 30, notch_angle + 30, white, -1)


draw_opencv_ring(canvas, red_center, red, 90)      # Top
draw_opencv_ring(canvas, green_center, green, 330)  # Bottom left
draw_opencv_ring(canvas, blue_center, blue, 270)     # Bottom right


# putText(image, text, org, font, fontScale, color , thickness , lineType, bottomLeftOrigin )
cv2.putText(canvas, "OpenCV", (130, 440), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4, lineType=cv2.LINE_AA)

# Save and display
cv2.imwrite("opencv_logo.jpg", canvas)
cv2.imshow("OpenCV Logo", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Q2

img1 = cv2.imread("./opencv_logo.jpg") 
img2 = cv2.imread("./dog.jpg") 


if img1 is None or img2 is None:
    print("Error: One or both image files not found.")
    exit()

# Resize 
img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))


# alpha is closed to 0 img1 is main, closed to 1 img2 is main
alpha = 0.6  

# Perform manual blending: blend = (1 - alpha) * img1 + alpha * img2
blend = ((1 - alpha) * img1 + alpha * img2)

# Save 
cv2.imwrite("manual_blend.jpg", blend)

# Display
cv2.imshow("Manual Blend Result", blend)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Q3

history_stack = []
action_log = []

def show_image(window, img):
    cv2.imshow(window, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# === Brightness ===
def adjust_brightness(img, value):
    new_img = cv2.convertScaleAbs(img, beta=value)
    action_log.append(f"brightness {value:+}")
    return new_img

# === Contrast ===
def adjust_contrast(img, alpha):
    new_img = cv2.convertScaleAbs(img, alpha=alpha)
    action_log.append(f"contrast x{alpha:.2f}")
    return new_img

# === Grayscale ===
def convert_grayscale(img):
    new_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
    action_log.append("converted to grayscale")
    return new_img

# === Padding with optional ratio ===
def add_padding(img, size, border_type, ratio_mode=None):
    h, w = img.shape[:2]

    if ratio_mode == "square":
        desired_ratio = 1.0
    elif ratio_mode == "rectangle":
        desired_ratio = 4 / 3 
    elif isinstance(ratio_mode, str) and ":" in ratio_mode:
        r_w, r_h = map(int, ratio_mode.split(":"))
        desired_ratio = r_w / r_h
    else:
        desired_ratio = w / h  # no ratio change

    # Match aspect ratio (minimal padding)
    current_ratio = w / h
    top = bottom = left = right = 0

    if abs(current_ratio - desired_ratio) > 1e-2:
        if current_ratio > desired_ratio:  # if too wide , add vertical padding
            new_h = int(w / desired_ratio)
            pad_h = (new_h - h) // 2
            top = bottom = pad_h
        else:  # if too tall , add horizontal padding
            new_w = int(h * desired_ratio)
            pad_w = (new_w - w) // 2
            left = right = pad_w

    # Add user-specified padding 
    top += size
    bottom += size
    left += size
    right += size

    border_map = {
        "constant": cv2.BORDER_CONSTANT,
        "reflect": cv2.BORDER_REFLECT,
        "replicate": cv2.BORDER_REPLICATE
    }

    new_img = cv2.copyMakeBorder(
        img, top, bottom, left, right,
        border_map.get(border_type, cv2.BORDER_CONSTANT), value=[0, 0, 0])

    action_log.append(f"padded ({ratio_mode or 'original'}) + {size}px")
    return new_img
# === Thresholding ===
def apply_threshold(img, mode):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh_type = cv2.THRESH_BINARY if mode == 'binary' else cv2.THRESH_BINARY_INV
    _, new_img = cv2.threshold(gray, 128, 255, thresh_type)
    new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
    action_log.append(f"threshold ({mode})")
    return new_img

# === Manual Blend ===
def blend_with_image(img1, path2, alpha):
    img2 = cv2.imread(path2)
    if img2 is None:
        print("Error: Cannot load second image.")
        return img1
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    blended = ((1 - alpha) * img1 + alpha * img2).astype(np.uint8)
    cv2.imwrite("manual_blend.jpg", blended)
    action_log.append(f"blended with {path2} at alpha={alpha}")
    return blended

# === Undo ===
def undo_last(img):
    if history_stack:
        prev = history_stack.pop()
        action_log.append("undo")
        return prev
    else:
        print("No previous state to undo.")
        return img

# === View log ===
def view_history():
    print("\n--- Operation History ---")
    for log in action_log:
        print(log)
    print("-------------------------\n")
    sys.stdout.flush() 

# === View Menu ===
def print_menu():
    print("""
==== Mini Photo Editor ====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose border type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image (manual alpha)
7. Undo Last Operation
8. View History of Operations
9. Save and Exit
""")

# === Load and Menu Loop ===
img_path = input("Enter the path to your image: ")
img = cv2.imread(img_path)

if img is None:
    print("Image not found.")
else:
    while True:
        
        print_menu()
        choice = input("Choose an option (1-9): ")

        if choice == "1":
            history_stack.append(img.copy())
            value = int(input("Enter brightness change (-100 to 100): "))
            img = adjust_brightness(img, value)
            clear_output(wait=True)
            show_image("Brightness", img)

        elif choice == "2":
            history_stack.append(img.copy())
            alpha = float(input("Enter contrast scale (0.0 = black, 0.5= duller, 1.0 = no change, 1.0> = sharper"))
            img = adjust_contrast(img, alpha)
            clear_output(wait=True)
            show_image("Contrast", img)

        elif choice == "3":
            history_stack.append(img.copy())
            img = convert_grayscale(img)
            clear_output(wait=True)
            show_image("Grayscale", img)

        elif choice == "4":
            history_stack.append(img.copy())
            size = int(input("Enter additional padding size (e.g., 10): "))
            border = input("Choose border type: (Border types: constant, reflect, replicate)").strip().lower()
            ratio_mode = input("Choose ratio mode: (Ratio modes: square, rectangle, custom, none)").strip().lower()

            if ratio_mode == "custom":
                ratio = input("Enter custom ratio (e.g., 4:5): ").strip()
            else:
                ratio = ratio_mode  # 'square', 'rectangle', or 'none'

            img = add_padding(img, size, border, ratio)
            clear_output(wait=True)
            show_image("Padding", img)

        elif choice == "5":
            history_stack.append(img.copy())
            mode = input("Threshold mode (binary/inverse): ").lower()
            img = apply_threshold(img, mode)
            clear_output(wait=True)
            show_image("Threshold", img)

        elif choice == "6":
            history_stack.append(img.copy())
            path2 = input("Enter second image path: ")
            alpha = float(input("Enter alpha (0 to 1): "))
            img = blend_with_image(img, path2, alpha)
            clear_output(wait=True)
            show_image("Blended", img)

        elif choice == "7":
            img = undo_last(img)
            clear_output(wait=True)
            show_image("Undo Result", img)

        elif choice == "8":
            view_history()

        elif choice == "9":
            filename = input("Enter filename to save (e.g., Example.jpg): ")
            cv2.imwrite(filename, img)
            print("Image saved. Exiting...")
            view_history()
            break

        else:
            print("Invalid option.")


==== Mini Photo Editor ====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose border type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image (manual alpha)
7. Undo Last Operation
8. View History of Operations
9. Save and Exit


--- Operation History ---
brightness -100
contrast x0.50
padded 5px reflect with ratio 4:4
undo
-------------------------


==== Mini Photo Editor ====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose border type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image (manual alpha)
7. Undo Last Operation
8. View History of Operations
9. Save and Exit

Invalid option.

==== Mini Photo Editor ====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose border type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image (manual alpha)
7. Undo Last Operation
8. View History of Operations
9. Save and Exit

Inval